## Map-Reduce

In [ ]:
l=[1,2,3,4]
l1 = list(map(lambda x: x*x, l))
print(l1)

[1, 4, 9, 16]


In [ ]:
from functools import reduce
prod = reduce(lambda x, y: x+y, l1)
print(prod)

30


In [ ]:
%%time
prod = reduce(lambda x, y: x+y, map(lambda x: x*x, l))
print(prod)

30
CPU times: user 53 µs, sys: 14 µs, total: 67 µs
Wall time: 70.6 µs


In [ ]:
%%time
l1 = list(map(lambda x: x*x, l))
prod = reduce(lambda x, y: x+y, l1)
print(prod)

30
CPU times: user 114 µs, sys: 0 ns, total: 114 µs
Wall time: 80.1 µs


In [ ]:
%%time
sum = 0
for li in l:
  sum = sum + (li*li)
print(sum)

30
CPU times: user 767 µs, sys: 0 ns, total: 767 µs
Wall time: 829 µs


## PySpark: RDD

In [2]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 73kB/s 
     |████████████████████████████████| 204kB 43.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=4babbd3c72d0d6fbb4e37df2b2bd30bb417dff30d70ab1fa2ba6f7bebb125013
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
import pyspark
from pyspark import SparkContext
sc = SparkContext("local", "first app")

In [4]:
#criando um RDD com uma lista de 0 a 3
A=sc.parallelize(range(4))

In [5]:
print(A)

PythonRDD[1] at RDD at PythonRDD.scala:53


In [6]:
print(A.collect())

[0, 1, 2, 3]


In [7]:
B = A.map(lambda x: x-2)
print(B.collect())

[-2, -1, 0, 1]


In [9]:
B = A.filter(lambda x: x%2==0)
B.collect()

[0, 2]

In [10]:
A=sc.parallelize(range(4))
A.reduce(lambda x,y: x+y)

6

In [11]:
# encontra o menor string
words=['this','is','the','best','mac','ever']
wordRDD=sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)<len(v) else v)

'is'

In [12]:
#dependente de ordem
B=sc.parallelize([1,3,5,2])
B.reduce(lambda x,y: x-y)

-9

In [13]:
%%time
B=sc.parallelize(range(4))
Squares=B.map(lambda x:x*x)
Squares.reduce(lambda x,y:x+y)

CPU times: user 11 ms, sys: 1.77 ms, total: 12.8 ms
Wall time: 198 ms


In [14]:
%%time
B.map(lambda x:x*x).reduce(lambda x,y:x+y)

CPU times: user 7.34 ms, sys: 2.11 ms, total: 9.44 ms
Wall time: 139 ms


14

In [15]:
n=100000
B=sc.parallelize([0,0,1,0]*n)
B.count()

400000

In [16]:
B.take(5)

[0, 0, 1, 0, 0]

In [17]:
#filtering
B.filter(lambda n: n > 0).count()

100000

In [18]:
#Removendo entradas repetidas
DuplicateRDD = sc.parallelize([1,1,2,2,3,3])
DistinctRDD = DuplicateRDD.distinct()
DistinctRDD.collect()

[1, 2, 3]

In [19]:
rdd1 = sc.parallelize([1, 1, 2, 3])
rdd2 = sc.parallelize([1, 3, 4, 5])

In [20]:
#Union
rdd1.union(rdd2).collect()

[1, 1, 2, 3, 1, 3, 4, 5]

In [21]:
#Interseccao
rdd1.intersection(rdd2).collect()

[1, 3]

## Examplo com texto

In [22]:
import urllib.request
with urllib.request.urlopen('https://mas-dse-open.s3.amazonaws.com/Moby-Dick.txt') as response:
   data = response.read()
    
text_file = open("Moby-Dick.txt", "w")
text_file.write(str(data))
text_file.close()    

In [23]:
text_file = sc.textFile("Moby-Dick.txt")
type(text_file)

pyspark.rdd.RDD

In [24]:
text_file.take(1)

['b\'\\xef\\xbb\\xbfThe Project Gutenberg EBook of Moby Dick; or The Whale, by Herman Melville\\r\\n\\r\\nThis eBook is for the use of anyone anywhere at no cost and with\\r\\nalmost no restrictions whatsoever.  You may copy it, give it away or\\r\\nre-use it under the terms of the Project Gutenberg License included\\r\\nwith this eBook or online at www.gutenberg.org\\r\\n\\r\\n\\r\\nTitle: Moby Dick; or The Whale\\r\\n\\r\\nAuthor: Herman Melville\\r\\n\\r\\nLast Updated: January 3, 2009\\r\\nPosting Date: December 25, 2008 [EBook #2701]\\r\\nRelease Date: June, 2001\\r\\n\\r\\nLanguage: English\\r\\n\\r\\n\\r\\n*** START OF THIS PROJECT GUTENBERG EBOOK MOBY DICK; OR THE WHALE ***\\r\\n\\r\\n\\r\\n\\r\\n\\r\\nProduced by Daniel Lazarus and Jonesey\\r\\n\\r\\n\\r\\n\\r\\n\\r\\n\\r\\nMOBY DICK; OR THE WHALE\\r\\n\\r\\nBy Herman Melville\\r\\n\\r\\n\\r\\n\\r\\n\\r\\nOriginal Transcriber\\\'s Notes:\\r\\n\\r\\nThis text is a combination of etexts, one from the now-defunct ERIS\\r\\nprojec

In [26]:
%%time
counts = text_file.flatMap(lambda line: line.split(" ")) \
.filter(lambda x: x!='')\
.map(lambda word: (word, 1)) \
.reduceByKey(lambda a, b: a + b)

CPU times: user 10.7 ms, sys: 1.11 ms, total: 11.8 ms
Wall time: 40.7 ms


In [27]:
type(counts)

pyspark.rdd.PipelinedRDD

In [28]:
#numero de palavras
counts.count()

44531

In [29]:
counts.take(5)

[("b'\\xef\\xbb\\xbfThe", 1),
 ('Project', 60),
 ('Gutenberg', 14),
 ('EBook', 1),
 ('of', 5784)]

In [30]:
%%time
c = counts.collect()

CPU times: user 25.9 ms, sys: 7.02 ms, total: 32.9 ms
Wall time: 232 ms


In [31]:
type(c)

list

In [ ]:
c[1:10]

[('Project', 60),
 ('Gutenberg', 14),
 ('EBook', 1),
 ('of', 5784),
 ('Moby', 68),
 ('Dick;', 7),
 ('or', 655),
 ('The', 389),
 ('Whale,', 29)]

In [32]:
%%time
c.sort(key=lambda x:x[1])

CPU times: user 8.74 ms, sys: 0 ns, total: 8.74 ms
Wall time: 8.75 ms


In [33]:
c[:5]

[("b'\\xef\\xbb\\xbfThe", 1),
 ('EBook', 1),
 ('Melville\\r\\n\\r\\nThis', 1),
 ('www.gutenberg.org\\r\\n\\r\\n\\r\\nTitle:', 1),
 ('Whale\\r\\n\\r\\nAuthor:', 1)]

In [34]:
c[-5:]

[('a', 3963), ('to', 3981), ('and', 5100), ('of', 5784), ('the', 11761)]

In [35]:
%%time
counts = text_file.flatMap(lambda line: line.split(" ")) \
.filter(lambda x: x!='')\
.map(lambda word: (word, 1)) \
.reduceByKey(lambda a, b: a + b)
c = counts.collect()
c.sort(key=lambda x:x[1])

CPU times: user 38.8 ms, sys: 3.88 ms, total: 42.7 ms
Wall time: 838 ms


In [36]:
c[-5:]

[('a', 3963), ('to', 3981), ('and', 5100), ('of', 5784), ('the', 11761)]

In [37]:
%%time
RDD=text_file.flatMap(lambda x: x.split(' '))\
.filter(lambda x: x!='')\
.map(lambda word: (word,1))
RDD1=RDD.reduceByKey(lambda x,y:x+y)
sorted_counts =  RDD1.sortBy(lambda x: x[1], True)
c1 = sorted_counts.collect()

CPU times: user 35.6 ms, sys: 5.16 ms, total: 40.8 ms
Wall time: 1.04 s


In [ ]:
c1[-5:]

[('a', 3963), ('to', 3981), ('and', 5100), ('of', 5784), ('the', 11761)]

## PySpark: Data Frames

In [38]:
#sqlcontext
from pyspark.sql import Row
from pyspark.sql import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


In [39]:
lista = [('John',19),('Smith',29),('Adam',35),('Henry',50)]

In [40]:
rdd = sc.parallelize(lista)

In [41]:
ppl = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
DF_ppl = sqlContext.createDataFrame(ppl)

In [42]:
DF_ppl.take(1)

[Row(name='John', age=19)]

In [43]:
DF_ppl.show(1)

+----+---+
|name|age|
+----+---+
|John| 19|
+----+---+
only showing top 1 row



In [44]:
DF_ppl.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [45]:
#from pyspark.sql import SQLContext
url = "https://raw.githubusercontent.com/ProfLuciano/cd/gh-pages/data/recife.csv"
from pyspark import SparkFiles

sc.addFile(url)
sqlContext = SQLContext(sc)

In [46]:
df = sqlContext.read.csv(SparkFiles.get("recife.csv"), header=True, inferSchema= True)			

In [47]:
df.printSchema()

root
 |-- price: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- area: double (nullable = true)
 |-- pkspaces: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- ensuites: integer (nullable = true)
 |-- suburb: string (nullable = true)
 |-- district: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- type: string (nullable = true)



In [48]:
df.show(5)

+------+----------+-----------+--------+----+--------+---------+--------+------------+--------+------+----------+-----+
| price|  latitude|  longitude|bedrooms|area|pkspaces|bathrooms|ensuites|      suburb|district|  city|     state| type|
+------+----------+-----------+--------+----+--------+---------+--------+------------+--------+------+----------+-----+
|  1500|-8.0597911|-34.8896191|       4|37.0|       3|        3|    null|   Boa Vista|  Recife|Recife|Pernambuco|apart|
|330000|-8.0295954|-34.9003716|       3|86.0|       1|        3|       1| Tamarineira|  Recife|Recife|Pernambuco|apart|
|255000|-8.0372467|-34.8909859|       2|82.0|       1|        2|    null|Encruzilhada|  Recife|Recife|Pernambuco|apart|
|380000|-8.0475622|-34.8769643|       3|95.0|       2|        3|       1| Santo Amaro|  Recife|Recife|Pernambuco|apart|
|300000|-8.0535802|-34.9087328|       3|97.0|       1|        3|       1|    Madalena|  Recife|Recife|Pernambuco|apart|
+------+----------+-----------+--------+

In [49]:
df.count()

1744

In [50]:
df.dropna().count()

758

In [ ]:
df.fillna(-1).show(5)

+------+----------+-----------+--------+----+--------+---------+--------+------------+--------+------+----------+-----+
| price|  latitude|  longitude|bedrooms|area|pkspaces|bathrooms|ensuites|      suburb|district|  city|     state| type|
+------+----------+-----------+--------+----+--------+---------+--------+------------+--------+------+----------+-----+
|  1500|-8.0597911|-34.8896191|       4|37.0|       3|        3|      -1|   Boa Vista|  Recife|Recife|Pernambuco|apart|
|330000|-8.0295954|-34.9003716|       3|86.0|       1|        3|       1| Tamarineira|  Recife|Recife|Pernambuco|apart|
|255000|-8.0372467|-34.8909859|       2|82.0|       1|        2|      -1|Encruzilhada|  Recife|Recife|Pernambuco|apart|
|380000|-8.0475622|-34.8769643|       3|95.0|       2|        3|       1| Santo Amaro|  Recife|Recife|Pernambuco|apart|
|300000|-8.0535802|-34.9087328|       3|97.0|       1|        3|       1|    Madalena|  Recife|Recife|Pernambuco|apart|
+------+----------+-----------+--------+

In [ ]:
df.select('price','area').show(5)

+------+----+
| price|area|
+------+----+
|  1500|37.0|
|330000|86.0|
|255000|82.0|
|380000|95.0|
|300000|97.0|
+------+----+
only showing top 5 rows



In [ ]:
df.groupBy("suburb").count().sort("count",ascending=False).show()

+------------+-----+
|      suburb|count|
+------------+-----+
|  Boa Viagem|  348|
| Santo Amaro|  131|
|Casa Amarela|  121|
| Tamarineira|   95|
|    Madalena|   89|
|      Gra̤as|   83|
|  Espinheiro|   80|
|       Torre|   77|
|  Casa Forte|   73|
|Encruzilhada|   69|
|     Aflitos|   65|
|  Parnamirim|   60|
|    Cordeiro|   40|
|    Jaqueira|   35|
|Campo Grande|   33|
|   Rosarinho|   28|
|        Po̤o|   25|
|   Boa Vista|   24|
| Imbiribeira|   22|
|       Prado|   21|
+------------+-----+
only showing top 20 rows



In [ ]:
df.describe().show()

+-------+-----------------+-------------------+--------------------+------------------+------------------+------------------+------------------+------------------+-------+--------+------+----------+-----+
|summary|            price|           latitude|           longitude|          bedrooms|              area|          pkspaces|         bathrooms|          ensuites| suburb|district|  city|     state| type|
+-------+-----------------+-------------------+--------------------+------------------+------------------+------------------+------------------+------------------+-------+--------+------+----------+-----+
|  count|             1744|               1744|                1744|              1737|              1721|              1646|              1538|               764|   1744|    1744|  1744|      1744| 1744|
|   mean| 465050.814793578| -8.059571519533849|  -34.90323247587737| 2.998272884283247|120.78249273678094|1.7885783718104495| 2.676853055916775|1.4293193717277486|   null|    null|

In [ ]:
df.filter(df.price > 30000).count()	

1220

In [ ]:
df1 = df.filter(df.price > 30000)

In [ ]:
df1.show(5)

+------+----------+-----------+--------+----+--------+---------+--------+------------+--------+------+----------+-----+
| price|  latitude|  longitude|bedrooms|area|pkspaces|bathrooms|ensuites|      suburb|district|  city|     state| type|
+------+----------+-----------+--------+----+--------+---------+--------+------------+--------+------+----------+-----+
|330000|-8.0295954|-34.9003716|       3|86.0|       1|        3|       1| Tamarineira|  Recife|Recife|Pernambuco|apart|
|255000|-8.0372467|-34.8909859|       2|82.0|       1|        2|    null|Encruzilhada|  Recife|Recife|Pernambuco|apart|
|380000|-8.0475622|-34.8769643|       3|95.0|       2|        3|       1| Santo Amaro|  Recife|Recife|Pernambuco|apart|
|300000|-8.0535802|-34.9087328|       3|97.0|       1|        3|       1|    Madalena|  Recife|Recife|Pernambuco|apart|
|320000|-8.0265163|-34.9177915|       2|50.0|       1|        2|       1|Casa Amarela|  Recife|Recife|Pernambuco|apart|
+------+----------+-----------+--------+

In [ ]:
from pyspark.sql.functions import *
df1 = df1.withColumn("price_m2", col("price")/col("area"))

In [ ]:
df1.show(5)

+------+----------+-----------+--------+----+--------+---------+--------+------------+--------+------+----------+-----+------------------+
| price|  latitude|  longitude|bedrooms|area|pkspaces|bathrooms|ensuites|      suburb|district|  city|     state| type|          price_m2|
+------+----------+-----------+--------+----+--------+---------+--------+------------+--------+------+----------+-----+------------------+
|330000|-8.0295954|-34.9003716|       3|86.0|       1|        3|       1| Tamarineira|  Recife|Recife|Pernambuco|apart|3837.2093023255816|
|255000|-8.0372467|-34.8909859|       2|82.0|       1|        2|    null|Encruzilhada|  Recife|Recife|Pernambuco|apart| 3109.756097560976|
|380000|-8.0475622|-34.8769643|       3|95.0|       2|        3|       1| Santo Amaro|  Recife|Recife|Pernambuco|apart|            4000.0|
|300000|-8.0535802|-34.9087328|       3|97.0|       1|        3|       1|    Madalena|  Recife|Recife|Pernambuco|apart| 3092.783505154639|
|320000|-8.0265163|-34.9177

In [ ]:
df1.printSchema()

root
 |-- price: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- area: double (nullable = true)
 |-- pkspaces: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- ensuites: integer (nullable = true)
 |-- suburb: string (nullable = true)
 |-- district: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- type: string (nullable = true)
 |-- price_m2: double (nullable = true)



In [ ]:
df1.groupBy("suburb").count().sort("count",ascending=True).show()

+--------------------+-----+
|              suburb|count|
+--------------------+-----+
|    Porto da Madeira|    1|
|         Mustardinha|    1|
|              Funḍo|    1|
|           Paissandu|    1|
|              Curado|    1|
|       Santo Ant̫nio|    1|
|             Coelhos|    1|
|            Est̢ncia|    1|
|         Dois Irṃos|    1|
|            Ṣo Jos̩|    2|
|          Mangabeira|    2|
|       Vasco da Gama|    2|
|             Torr̵es|    2|
|               Zumbi|    2|
|           Pau-Ferro|    2|
|               Cohab|    3|
|Cidade Universit�...|    3|
|               Ibura|    3|
|            Tejipi�_|    3|
|              Areias|    3|
+--------------------+-----+
only showing top 20 rows



In [ ]:
df_remove = df1.filter(df1.suburb !=	'Mustardinha')

In [ ]:
df_remove.groupBy("suburb").count().sort("count",ascending=True).show()

+----------------+-----+
|          suburb|count|
+----------------+-----+
|         Torr̵es|    1|
|         Coelhos|    1|
|          Funḍo|    1|
|   Santo Ant̫nio|    1|
|Porto da Madeira|    1|
|        Tejipi�_|    1|
|        Ṣo Jos̩|    2|
|       Pau-Ferro|    2|
|      Mangabeira|    2|
|         V��rzea|    2|
|   Vasco da Gama|    2|
|           Ibura|    3|
|          Areias|    3|
|           Cohab|    3|
| Engenho do Meio|    3|
| Ponto de Parada|    4|
|           Derby|    4|
|           Ipsep|    5|
|         Santana|    5|
|        Iputinga|    6|
+----------------+-----+
only showing top 20 rows



In [ ]:
#usando consultas SQL
df.registerTempTable('df_table')

In [ ]:
sqlContext.sql('select * from df_table where suburb = "Tamarineira" ').show(5)

+------+----------+-----------+--------+-----+--------+---------+--------+-----------+--------+------+----------+-----+
| price|  latitude|  longitude|bedrooms| area|pkspaces|bathrooms|ensuites|     suburb|district|  city|     state| type|
+------+----------+-----------+--------+-----+--------+---------+--------+-----------+--------+------+----------+-----+
|330000|-8.0295954|-34.9003716|       3| 86.0|       1|        3|       1|Tamarineira|  Recife|Recife|Pernambuco|apart|
|360000|-8.0331118|-34.8974905|       2| 60.0|       1|        2|       1|Tamarineira|  Recife|Recife|Pernambuco|apart|
|370000|-8.0331118|-34.8974905|       3| 72.0|       1|        3|       1|Tamarineira|  Recife|Recife|Pernambuco|apart|
|320000|-8.0331118|-34.8974905|       2| 55.0|       1|        2|       1|Tamarineira|  Recife|Recife|Pernambuco|apart|
|500000|-8.0331118|-34.8974905|       3|100.0|       2|        3|       1|Tamarineira|  Recife|Recife|Pernambuco|apart|
+------+----------+-----------+--------+

In [ ]:
sqlContext.sql('select suburb, mean(price) from df_table group by suburb').show()

+--------------------+------------------+
|              suburb|        avg(price)|
+--------------------+------------------+
|            Monteiro|          713676.9|
|          Espinheiro|       536896.2375|
|                Po̤o|         890523.64|
|               Prado| 320601.3333333333|
|            Est̢ncia|             600.0|
|        Campo Grande| 303883.9696969697|
|    Jardim Ṣo Paulo|             360.6|
|            Iputinga| 101318.8947368421|
|               Torre|346458.71428571426|
|            Soledade|1108790.2857142857|
|              Arruda|319677.85714285716|
|         Mustardinha|             750.0|
|               Zumbi|             850.0|
|Cidade Universit�...|4081.6666666666665|
|            Jaqueira| 935032.2285714286|
|         Imbiribeira| 682272.8181818182|
|           Boa Vista|131135.83333333334|
|               Derby| 731045.5714285715|
|      Ilha do Retiro| 571205.5555555555|
|            Cordeiro|        345850.675|
+--------------------+------------